In [4]:
# import pandas as pd
# import modin as pd
import boto3
import pandas as pd
import s3fs
import re
import logging

from botocore.exceptions import ClientError


In [5]:
post_path = "../Data/wallstreetbets_posts.csv"
comment_path = "../Data/wallstreetbets_comments.csv"

In [6]:
comment_df = pd.read_csv(comment_path)
post_df = pd.read_csv(post_path)

In [7]:
comment_df.columns.to_list()

['all_awardings',
 'associated_award',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'body',
 'collapsed_because_crowd_control',
 'comment_type',
 'created_utc',
 'gildings',
 'id',
 'is_submitter',
 'link_id',
 'locked',
 'no_follow',
 'parent_id',
 'permalink',
 'retrieved_on',
 'score',
 'send_replies',
 'stickied',
 'subreddit',
 'subreddit_id',
 'top_awarded_type',
 'total_awards_received',
 'treatment_tags',
 'distinguished',
 'author_cakeday']

In [9]:
post_df.columns.to_list()

['all_awardings',
 'allow_live_comments',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'domain',
 'full_link',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'pwls',
 'retrieved_on',
 'score',
 'selftext',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 '

In [10]:
comment_df.isnull().sum()

all_awardings                            0
associated_award                   9559657
author                                   0
author_flair_background_color      9392288
author_flair_css_class             9554484
author_flair_richtext              2357655
author_flair_template_id           9392288
author_flair_text                  8581524
author_flair_text_color            6223708
author_flair_type                  2357655
author_fullname                    2357655
author_patreon_flair               2357655
author_premium                     2357655
awarders                                 0
body                                    10
collapsed_because_crowd_control    9559657
comment_type                       9559657
created_utc                              0
gildings                                 0
id                                       0
is_submitter                             0
link_id                                  0
locked                                   0
no_follow  

In [11]:
len(comment_df)

9559657

In [12]:
comment_df["author"].sample(50)

2536127         michaelmfjordan
287063      Head_Paleontologist
5573926               [deleted]
8421834             MrCumberbum
3819306          UselessPresent
3773270               Nehkrosis
3393342               potspands
8086989               [deleted]
5409839               [deleted]
2945199                  hilljz
9441031    ShoddyLetterhead3491
4884064               [deleted]
732045        Occams_shave_club
568421       PM_Me_Ur_Greyhound
5037755               [deleted]
8339408                Viisionn
7121950               [deleted]
7228407               rsmtirish
6258353                Matt_M_3
8320389               [deleted]
4986284               [deleted]
1481428               realister
3468877        DeadlyPantsOfSea
833845     Lazy_Willingness_420
7826979               [deleted]
8364920                  Tkuhug
2643689           OldManWisdom3
1233075               [deleted]
3416487            No-Math-5079
9383982              redcedar53
5663460              NorthWind_
8618697 

In [13]:
comment_df["author"].value_counts()

[deleted]             2357655
AutoModerator           72804
PlaneIntroduction8      10237
OptionsAndTren           6576
WSBVoteBot               6397
                       ...   
BigFatEgg                   1
SteezyStiz                  1
im-jam                      1
SnooCupcakes6438            1
imstrobey                   1
Name: author, Length: 657292, dtype: int64

In [14]:
cleaned_df = comment_df[comment_df["author"] != "[deleted]"]

In [15]:
cleaned_df

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,score,send_replies,stickied,subreddit,subreddit_id,top_awarded_type,total_awards_received,treatment_tags,distinguished,author_cakeday
0,[],NaN,guesswhatihate,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
1,[],NaN,k12nmonky,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
2,[],NaN,tnmtnmtnm,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
3,[],NaN,everlastingdeath,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
4,[],NaN,ZakkuTurner,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559652,[],NaN,AngryBobRoss,NaN,NaN,[],NaN,NaN,NaN,text,...,1,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
9559653,[],NaN,Machin_Shins,NaN,NaN,[],NaN,NaN,NaN,text,...,2,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
9559654,[],NaN,testfire10,NaN,NaN,[],NaN,NaN,NaN,text,...,22,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN
9559655,[],NaN,stupidlysweet,NaN,NaN,[],NaN,NaN,NaN,text,...,12,True,False,wallstreetbets,t5_2th52,NaN,0,[],NaN,NaN


In [16]:
cleaned_df = cleaned_df.drop([ 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'collapsed_because_crowd_control',
 'comment_type', 
 'locked',
 'no_follow', 
 'all_awardings',
 'associated_award', 
 'send_replies', 
 'stickied', 
 'subreddit', 
 'subreddit_id', 
 'distinguished', 
 'author_cakeday'],1)

In [17]:
cleaned_df

,author,awarders,body,created_utc,gildings,id,is_submitter,link_id,parent_id,permalink,retrieved_on,score,top_awarded_type,total_awards_received,treatment_tags
0,guesswhatihate,[],#DIPPING,1607957482,{},gftdfol,False,t3_kcvkwx,t1_gftdcb8,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
1,k12nmonky,[],NIO and SPCE ftw,1607957482,{},gftdfo4,False,t3_kcvkwx,t1_gftdc1s,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
2,tnmtnmtnm,[],Flat,1607957482,{},gftdfo3,False,t3_kcvkwx,t1_gftdbth,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
3,everlastingdeath,[],Lol what happened to the limit up everyone's b...,1607957481,{},gftdfnp,False,t3_kcvkwx,t3_kcvkwx,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957494,1,NaN,0,[]
4,ZakkuTurner,[],"Jan 22 2021, 40.5",1607957481,{},gftdfmf,False,t3_kcvkwx,t1_gftddng,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1607957493,1,NaN,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559652,AngryBobRoss,[],Lmao I got 2 nights to sleep over it. I might ...,1611412578,{},gkdmw91,False,t3_l2shj5,t1_gkceomw,/r/wallstreetbets/comments/l2shj5/doing_my_par...,1611571732,1,NaN,0,[]
9559653,Machin_Shins,[],Squeeze aside. Are there price targets for GME...,1611412578,{},gkdmw6k,False,t3_l3c0hj,t3_l3c0hj,/r/wallstreetbets/comments/l3c0hj/considering_...,1611571731,2,NaN,0,[]
9559654,testfire10,[],TD Ameritrade,1611412576,{},gkdmw1y,False,t3_l3cp2s,t1_gkdloa8,/r/wallstreetbets/comments/l3cp2s/gme_gainz_so...,1611571730,22,NaN,0,[]
9559655,stupidlysweet,[],Same. I've got 30k in savings but thinking of ...,1611412575,{},gkdmvxa,False,t3_l35s4w,t1_gkbyl29,/r/wallstreetbets/comments/l35s4w/do_not_sell_...,1611571728,12,NaN,0,[]


In [18]:
cleaned_df = cleaned_df.drop(['top_awarded_type','retrieved_on'],1)

In [19]:
cleaned_df

,author,awarders,body,created_utc,gildings,id,is_submitter,link_id,parent_id,permalink,score,total_awards_received,treatment_tags
0,guesswhatihate,[],#DIPPING,1607957482,{},gftdfol,False,t3_kcvkwx,t1_gftdcb8,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0,[]
1,k12nmonky,[],NIO and SPCE ftw,1607957482,{},gftdfo4,False,t3_kcvkwx,t1_gftdc1s,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0,[]
2,tnmtnmtnm,[],Flat,1607957482,{},gftdfo3,False,t3_kcvkwx,t1_gftdbth,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0,[]
3,everlastingdeath,[],Lol what happened to the limit up everyone's b...,1607957481,{},gftdfnp,False,t3_kcvkwx,t3_kcvkwx,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0,[]
4,ZakkuTurner,[],"Jan 22 2021, 40.5",1607957481,{},gftdfmf,False,t3_kcvkwx,t1_gftddng,/r/wallstreetbets/comments/kcvkwx/daily_discus...,1,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9559652,AngryBobRoss,[],Lmao I got 2 nights to sleep over it. I might ...,1611412578,{},gkdmw91,False,t3_l2shj5,t1_gkceomw,/r/wallstreetbets/comments/l2shj5/doing_my_par...,1,0,[]
9559653,Machin_Shins,[],Squeeze aside. Are there price targets for GME...,1611412578,{},gkdmw6k,False,t3_l3c0hj,t3_l3c0hj,/r/wallstreetbets/comments/l3c0hj/considering_...,2,0,[]
9559654,testfire10,[],TD Ameritrade,1611412576,{},gkdmw1y,False,t3_l3cp2s,t1_gkdloa8,/r/wallstreetbets/comments/l3cp2s/gme_gainz_so...,22,0,[]
9559655,stupidlysweet,[],Same. I've got 30k in savings but thinking of ...,1611412575,{},gkdmvxa,False,t3_l35s4w,t1_gkbyl29,/r/wallstreetbets/comments/l35s4w/do_not_sell_...,12,0,[]


In [20]:
# cleaned_df.to_csv("cleaned_wsb_comments.csv", index=False)

In [21]:
client = boto3.client('s3')

In [22]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('wsb-final-project-bucket')
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='aws-programmatic-access-test-object')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='kaggle_wsb_comments_12:6-2:6.csv')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='kaggle_wsb_posts_12:6-2:6.csv')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/b1a0304a-737f-4cc0-bff8-a7911ddf79ba/test-1.csv.gz')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/b1a0304a-737f-4cc0-bff8-a7911ddf79ba/test-Manifest.json')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210101-20210201/test-Manifest.json')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210201-20210301/305336a5-a905-4bf1-8b81-2f5d18953f59/test-1.csv.gz')
s3.ObjectSummary(bucket_name='wsb-final-project-bucket', key='test/test/20210201-20210301/305336a5-a905-4bf1-8b81-2f5d18953f59/test-Manifest.json')
s3.ObjectSummary

In [23]:
# response = client.upload_file("../Data/cleaned_wsb_comments.csv", "wsb-final-project-bucket", "cleaned_wsb_comments.csv")

In [59]:
post_df

,all_awardings,allow_live_comments,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,...,secure_media_embed,banned_by,author_flair_template_id,author_cakeday,edited,gilded,distinguished,collections,crosspost_parent,crosspost_parent_list
0,[],False,readyrummy1,NaN,NaN,"[{'e': 'text', 't': '\x1b201202:4:1'}]",201202:4:1,dark,richtext,t2_7ppu74oe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,Pluto_Muto,NaN,NaN,[],NaN,NaN,text,t2_9bo9tvhh,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Hiro08,NaN,NaN,[],NaN,NaN,text,t2_17panrff,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,uslashuname,NaN,NaN,[],NaN,NaN,text,t2_3nee5zpr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,dadwhomstfucks12,NaN,NaN,"[{'e': 'text', 't': '\x1b201202:3:1'}]",201202:3:1,dark,richtext,t2_50baj197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707456,[],False,Leather-Clock1917,NaN,NaN,"[{'e': 'text', 't': '\x1b201210:2:1'}]",201210:2:1,dark,richtext,t2_3n0fymdu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707457,[],False,TBJumbo,NaN,NaN,[],NaN,NaN,text,t2_8du4uqjl,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707458,[],False,Thatcrazygoatman,NaN,NaN,[],NaN,NaN,text,t2_o0rlwa4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
707459,[],False,ZookeepergameSoggy49,NaN,NaN,[],NaN,NaN,text,t2_80xn5ah7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
post_df["author"].value_counts()

[deleted]               57916
dhiral1994                121
PubertEHumphrey           107
CappedCrib                100
AutoModerator              98
                        ...  
Sissyslave649               1
deux08                      1
eamonnamro                  1
Scary_Victory               1
Expensive-Sense-2438        1
Name: author, Length: 388208, dtype: int64

In [27]:
post_df.columns.to_list()

['all_awardings',
 'allow_live_comments',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'domain',
 'full_link',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'pwls',
 'retrieved_on',
 'score',
 'selftext',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 '

In [60]:
garbage_columns = ['author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'domain',
 'gildings',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'over_18',
 'parent_whitelist_status',
 'pinned',
 'pwls',
 'retrieved_on',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 'suggested_sort',
 'thumbnail',
 'treatment_tags',
 'whitelist_status',
 'wls',
 'removed_by_category',
 'post_hint',
 'preview',
 'thumbnail_height',
 'thumbnail_width',
 'url_overridden_by_dest',
 'media_metadata',
 'gallery_data',
 'is_gallery',
 'media',
 'media_embed',
 'secure_media',
 'secure_media_embed',
 'banned_by',
 'author_flair_template_id',
 'author_cakeday',
 'edited',
 'gilded',
 'distinguished',
 'collections',
 'crosspost_parent',
 'crosspost_parent_list',
 'allow_live_comments',
 'num_crossposts',
 'all_awardings'
 ]

In [61]:
clean_post_df = post_df.drop(garbage_columns,1)

In [62]:
clean_post_df

,author,created_utc,full_link,id,num_comments,permalink,score,selftext,title,total_awards_received,upvote_ratio,url
0,readyrummy1,1607957430,https://www.reddit.com/r/wallstreetbets/commen...,kcz0hf,6,/r/wallstreetbets/comments/kcz0hf/ruled_by_the...,1,5:30 AM\n\nWoke up five minutes ago. Made som...,Ruled by the Theta Gods,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
1,Pluto_Muto,1607957404,https://www.reddit.com/r/wallstreetbets/commen...,kcz07r,0,/r/wallstreetbets/comments/kcz07r/3k_and_i_wan...,1,[removed],3k and I want more,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
2,Hiro08,1607957384,https://www.reddit.com/r/wallstreetbets/commen...,kcyzzz,0,/r/wallstreetbets/comments/kcyzzz/thank_you_fe...,1,NaN,Thank you fellow retards and autists! Big ston...,0,1.0,https://i.redd.it/i6ae8w4c16561.jpg
3,uslashuname,1607957282,https://www.reddit.com/r/wallstreetbets/commen...,kcyyw4,0,/r/wallstreetbets/comments/kcyyw4/azn_fair_val...,1,[removed],"AZN fair value of $55, price of $50, vaccine c...",0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
4,dadwhomstfucks12,1607957280,https://www.reddit.com/r/wallstreetbets/commen...,kcyyve,0,/r/wallstreetbets/comments/kcyyve/i_wanna_be_l...,1,[removed],I wanna be less retarded,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
...,...,...,...,...,...,...,...,...,...,...,...,...
707456,Leather-Clock1917,1609093756,https://www.reddit.com/r/wallstreetbets/commen...,kl8lcn,15,/r/wallstreetbets/comments/kl8lcn/bro_this_is_...,1,NaN,bro this is the most true shit ever and i can’...,0,1.0,https://i.redd.it/mwk4t0tcwr761.jpg
707457,TBJumbo,1609093502,https://www.reddit.com/r/wallstreetbets/commen...,kl8ink,15,/r/wallstreetbets/comments/kl8ink/took_5_years...,1,NaN,Took 5 years of losing on trading shares until...,0,1.0,https://i.redd.it/rgt6krklvr761.jpg
707458,Thatcrazygoatman,1609093325,https://www.reddit.com/r/wallstreetbets/commen...,kl8gni,6,/r/wallstreetbets/comments/kl8gni/_/,1,NaN,💎 ✋,0,1.0,https://i.redd.it/hazc4oh2vr761.jpg
707459,ZookeepergameSoggy49,1609093189,https://www.reddit.com/r/wallstreetbets/commen...,kl8f6l,18,/r/wallstreetbets/comments/kl8f6l/was_it_a_dre...,1,Large GameStop Investor Ryan Cohen Buys More S...,Was it a dream??!😴 😴 GME opens on Monday up 100%,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...


In [63]:
clean_post_df = clean_post_df[clean_post_df["author"] != "[deleted]"]

In [64]:
clean_post_df

,author,created_utc,full_link,id,num_comments,permalink,score,selftext,title,total_awards_received,upvote_ratio,url
0,readyrummy1,1607957430,https://www.reddit.com/r/wallstreetbets/commen...,kcz0hf,6,/r/wallstreetbets/comments/kcz0hf/ruled_by_the...,1,5:30 AM\n\nWoke up five minutes ago. Made som...,Ruled by the Theta Gods,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
1,Pluto_Muto,1607957404,https://www.reddit.com/r/wallstreetbets/commen...,kcz07r,0,/r/wallstreetbets/comments/kcz07r/3k_and_i_wan...,1,[removed],3k and I want more,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
2,Hiro08,1607957384,https://www.reddit.com/r/wallstreetbets/commen...,kcyzzz,0,/r/wallstreetbets/comments/kcyzzz/thank_you_fe...,1,NaN,Thank you fellow retards and autists! Big ston...,0,1.0,https://i.redd.it/i6ae8w4c16561.jpg
3,uslashuname,1607957282,https://www.reddit.com/r/wallstreetbets/commen...,kcyyw4,0,/r/wallstreetbets/comments/kcyyw4/azn_fair_val...,1,[removed],"AZN fair value of $55, price of $50, vaccine c...",0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
4,dadwhomstfucks12,1607957280,https://www.reddit.com/r/wallstreetbets/commen...,kcyyve,0,/r/wallstreetbets/comments/kcyyve/i_wanna_be_l...,1,[removed],I wanna be less retarded,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...
...,...,...,...,...,...,...,...,...,...,...,...,...
707456,Leather-Clock1917,1609093756,https://www.reddit.com/r/wallstreetbets/commen...,kl8lcn,15,/r/wallstreetbets/comments/kl8lcn/bro_this_is_...,1,NaN,bro this is the most true shit ever and i can’...,0,1.0,https://i.redd.it/mwk4t0tcwr761.jpg
707457,TBJumbo,1609093502,https://www.reddit.com/r/wallstreetbets/commen...,kl8ink,15,/r/wallstreetbets/comments/kl8ink/took_5_years...,1,NaN,Took 5 years of losing on trading shares until...,0,1.0,https://i.redd.it/rgt6krklvr761.jpg
707458,Thatcrazygoatman,1609093325,https://www.reddit.com/r/wallstreetbets/commen...,kl8gni,6,/r/wallstreetbets/comments/kl8gni/_/,1,NaN,💎 ✋,0,1.0,https://i.redd.it/hazc4oh2vr761.jpg
707459,ZookeepergameSoggy49,1609093189,https://www.reddit.com/r/wallstreetbets/commen...,kl8f6l,18,/r/wallstreetbets/comments/kl8f6l/was_it_a_dre...,1,Large GameStop Investor Ryan Cohen Buys More S...,Was it a dream??!😴 😴 GME opens on Monday up 100%,0,1.0,https://www.reddit.com/r/wallstreetbets/commen...


In [66]:
clean_post_df.to_csv("cleaned_wsb_posts.csv", index=False)